In [162]:
#Numba#

In [6]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [7]:
cap = cv2.VideoCapture('bird.avi')
 
while(True):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.0.0) /io/opencv/modules/imgproc/src/color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [1]:
from numba import njit,jit
import numpy as np
import cv2
import itertools

cap = cv2.VideoCapture('bird.avi')

quesize = 10
stabilityqueue = np.zeros((480,640,quesize,3),dtype=np.int32)
w = np.zeros((480,640),dtype=np.int32)
ID = np.zeros((480,640),dtype=np.int32)
queueLimits = np.zeros((480,640,2),dtype=np.int32)
#print(stabilityqueue[:,:,0,:].shape)

@njit
def IsFoundinStabilityQueue(pixel,i,j):
    for queueIndex in range(queueLimits[i,j,0],queueLimits[i,j,1]):
        queuevalue = np.reshape(stabilityqueue[i,j,queueIndex,:],(3,1))
        if queuevalue[0][0]==pixel[0][0] and queuevalue[1][0]==pixel[1][0] and queuevalue[2][0]==pixel[2][0]:
            return True
    return False
    
@njit
def bewis(frame):
    ID = np.zeros((480,640),dtype=np.int32)
    w = np.zeros((480,640),dtype=np.int32)
    for i in range(frame.shape[0]):
        for j in range(frame.shape[1]):
            r,g,b = frame[i,j,:]
            #create binary endoder for each pixel
            a = np.zeros((256,3))
            a[0:r,0] = 1
            a[0:g,1] = 1
            a[0:b,2] = 1
            #check if pixel color is in front of stability queue
            front = np.reshape(stabilityqueue[i,j,queueLimits[i,j,0],:],(3,1)) 
            pixel = np.reshape(frame[i,j,:],(3,1))
            if front[0][0]==pixel[0][0] and front[1][0]==pixel[1][0] and front[2][0]==pixel[2][0]:
                w[i][j] = ID[i][j]
                ID[i,j] += 1
            elif IsFoundinStabilityQueue(pixel,i,j):
                x=0

                            

while(True):
        
    ret, frame = cap.read()
    # Display the resulting frame
    cv2.imshow('frame',frame)
    
    bewis(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


'''
    print(frame.shape)
    for p in itertools.product(range(frame.shape[0]),range(frame.shape[1])):
        print (p)
        #r,g,b = frame[p[0],p[1],:]
        #a = np.zeros((256,3))
        #a[0:r,0] = 1  
        #a[0:g,1] = 1
        #a[0:b,2] = 1
'''

'\n    print(frame.shape)\n    for p in itertools.product(range(frame.shape[0]),range(frame.shape[1])):\n        print (p)\n        #r,g,b = frame[p[0],p[1],:]\n        #a = np.zeros((256,3))\n        #a[0:r,0] = 1  \n        #a[0:g,1] = 1\n        #a[0:b,2] = 1\n'

In [3]:
import time
from numba import njit,prange
import numpy as np
import random
import tensorflow as tf

In [4]:
def preprocessing(tx_train, ty_train, tx_test, ty_test):
        
    py_train = ty_train.flatten()
    py_test = ty_test.flatten()
    
    px_test = tx_test.reshape(10000,input_size)
    px_train = tx_train.reshape(60000,input_size)
    
    px_train = np.asarray(px_train).astype(np.int32)
    py_train = np.asarray(py_train).astype(np.int32)
    px_test = np.asarray(px_test).astype(np.int32)
    py_test = np.asarray(py_test).astype(np.int32)

    return px_train, py_train, px_test, py_test   


In [5]:
class WiSARD:                                                     
    
    def __init__(self,input_size,no_of_rand_pix_selec,nodes,ram_address_count,dis_number):
        self.input_size = input_size
        self.no_of_rand_pix_selec = no_of_rand_pix_selec
        self.nodes = nodes
        self.ram_address_count = ram_address_count
        self.dis_number = dis_number 


    def discriminator(self):
        discriminator = []
        accumulated_pos = []
        my_list = list(range(0,self.input_size))
        for i in range(self.dis_number):  #10
            ram = []
            random.shuffle(my_list)
            for j in range((int)((self.nodes))): #98    
                total_pos = []            
                positions = []
                positions = my_list[j*no_of_rand_pix_selec:j*no_of_rand_pix_selec+no_of_rand_pix_selec]
                accumulated_pos.append(positions)
                total_pos = np.vstack(positions)
                #print(total_pos)
                table = []
                dictionary = {}
                
                max = len("{0:b}".format(2**len(total_pos))) - 1
                for i in range(2**len(total_pos)):
                    x = (('0' * max) + "{0:b}".format(i))
                    x = x[len(x)-max:]
                    dictionary[x] = 0
                table.append(dictionary)
                #print(table)
                ram.append(table)
        
            di = []
            for j in range(len(ram)):
                for i in range(len(ram[j])):
                    for key, value in ram[j][i].items():
                        temp = [key,value]
                        di.append(temp)
                        
            discriminator.append(di)
    
        discriminator = np.asarray(discriminator).astype(np.int32)
        accumulated_pos = np.asarray(accumulated_pos).astype(np.int32)
        return discriminator, accumulated_pos
    
                        
    
    @staticmethod
    @njit(parallel = True)
    def train_with_bleeching(d,pos,x_train, y_train):
        
        images = x_train
        lable = y_train    
        
        for i in range(len(images)):
            image = images[i]
            num = lable[i]
            all_ram_of_selected_discriminator = d[num]
            t_ratina = pos[(int)(nodes*num):(int)(nodes*num+nodes)]
            
            for i in range((int)(nodes)):
                part = all_ram_of_selected_discriminator[(ram_address_count*i):(ram_address_count*i+ram_address_count)]
                ratina_for_one_ram = t_ratina[i]
                
                #threshold = 0               
                n = []                                                                
                for ix in range(len(ratina_for_one_ram)):
                    pix = ratina_for_one_ram[ix]
                    if image[(pix-1)]>=1:
                        n.append(1)
                        #threshold = threshold + 1     
                    else:
                        n.append(0)
                
                num = 0
                for i in range(no_of_rand_pix_selec):
                    num = (n[i])*(10**((no_of_rand_pix_selec-1)-i)) + num
            
                address_of_that_ram = (int)(num)
                for key in range(ram_address_count):
                    index = part[key]
                    #if threshold >= 1:          
                    if index[0] == address_of_that_ram:
                        index[1] += 1
                    else:
                        if index[1] != 0:
                            index[1] -= 1



    #@vectorize(['int32(int32,int32,int32,int32)'], target = 'cuda')
    @staticmethod
    @njit(parallel = True)
    def test_with_bleaching(d,pos,x_test,y_test):
        right = 0
        wrong = 0
        images = x_test
        lable = y_test
        ct = 0.02
        b=1
        
        for i in range(len(images)):
            image = images[i]
            actual_lable = lable[i]
            
            total_sum=[]
            
            for ix in range(dis_number):
                
                t_ratina = pos[int((nodes*ix)):(int)((nodes*ix+nodes))]
                
                sum_of_ram_output = 0
                dis = d[ix]
                
                for i in range(int(nodes)):
                    part = dis[(ram_address_count*i):(ram_address_count*i+ram_address_count)]
                    ratina_for_one_ram = t_ratina[i]
                    
                    n = []                                                                
                    for pix in ratina_for_one_ram:
                        if image[(pix-1)]>=1:
                            n.append(1)
                        else:
                            n.append(0)
                    
                    num = 0
                    for i in range(no_of_rand_pix_selec):
                        num = (n[i])*(10**((no_of_rand_pix_selec-1)-i)) + num
                    
                    address_of_that_ram = (int)(num)
                
                    for key in range(len(part)):
                        prt = part[key]
                        if prt[0] == address_of_that_ram and prt[1]>=b:           #have to change this
                            sum_of_ram_output += 1
                
                total_sum.append(sum_of_ram_output)        
        
            max_sum = 0
            sec_max = 0
            idx = 0
            
            for i in range(len(total_sum)):
                if max_sum < total_sum[i]:
                    max_sum = total_sum[i]
                    idx = i
                    
            for j in range(len(total_sum)):
                if sec_max < total_sum[j] and j!=idx:
                    sec_max = total_sum[j]
                    
            index_of_dis = idx
            if index_of_dis == actual_lable:
                right += 1
            else:
                wrong += 1
            
            if max_sum == sec_max:
                confidence = 0
            else:
                confidence = 1 - float(sec_max)/float(max_sum)
                if confidence < ct:
                    b += 1
        
        return right,wrong

In [6]:
input_size = 28*28
no_of_rand_pix_selec = 2**(3)     ## ** (must) no_of_rand_pix_selec = 2^(n) where n is 0,1,2...
nodes = int(input_size/no_of_rand_pix_selec)    #98
ram_address_count = 2**(no_of_rand_pix_selec)   #256
dis_number = 10                #10 i.e number of lables

In [7]:
(tx_train, ty_train), (tx_test, ty_test) = tf.keras.datasets.mnist.load_data()
px_train, py_train, px_test, py_test = preprocessing(tx_train, ty_train, tx_test, ty_test)

In [8]:
w = WiSARD(input_size,no_of_rand_pix_selec,nodes,ram_address_count,dis_number)
d, acc_pos = w.discriminator()
print(d.shape)

(10, 25088, 2)


In [9]:
starttrain = time.time()
w.train_with_bleeching(d,acc_pos,px_train[0:60000],py_train[0:60000])
endtrain = time.time()
print("time train = ",endtrain - starttrain)

time train =  19.105674028396606


In [10]:
starttest = time.time()
right,wrong = w.test_with_bleaching(d,acc_pos,px_test[0:10000],py_test[0:10000])
endtest = time.time()
print("time test = ",endtest - starttest)

time test =  31.591220378875732


In [11]:
print("number of right result = ",right)
print("number of wrong results = ",wrong)
accuracy = ((right)/(right+wrong))*100
print("accuracy by testing the model =",accuracy)
#    end = time.time()
#    print("total time = ",end - start)

number of right result =  5612
number of wrong results =  4388
accuracy by testing the model = 56.120000000000005
